In [ ]:
pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 8.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

In [ ]:
pip install pandas


In [ ]:
pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 22.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
pip install datasets

In [ ]:
pip install spacy

In [ ]:
pip install nltk

In [ ]:
import stanza
from datasets import *
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/NYT_Dataset.csv",index_col = 0, parse_dates = ["Date"],na_values=['nan'])
train, test = train_test_split(df, test_size=0.2)

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
d = {}
for x in train['title']:
  doc = nlp(x)
  print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')


In [ ]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
import random
text_data = []
for x in train['title']:
  tokens = prepare_text_for_lda(x)
  text_data.append(tokens)

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.038*"attack" + 0.036*"kill" + 0.025*"opposition" + 0.024*"clash"')
(1, '0.031*"police" + 0.025*"mideast" + 0.024*"near" + 0.021*"force"')
(2, '0.042*"strike" + 0.032*"pentagon" + 0.027*"afghanistan" + 0.022*"soldier"')
(3, '0.035*"official" + 0.030*"south" + 0.026*"election" + 0.025*"korea"')
(4, '0.031*"france" + 0.027*"russia" + 0.027*"death" + 0.020*"israel"')
(5, '0.037*"leader" + 0.028*"seek" + 0.024*"suharto" + 0.023*"britain"')
(6, '0.024*"mexico" + 0.022*"british" + 0.021*"return" + 0.021*"government"')
(7, '0.040*"zimbabwe" + 0.031*"iraqi" + 0.028*"press" + 0.026*"leaders"')
(8, '0.048*"palestinian" + 0.037*"talks" + 0.028*"report" + 0.022*"chief"')
(9, '0.035*"china" + 0.025*"ground" + 0.021*"kill" + 0.021*"pakistan"')


In [ ]:
# Steps
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.parse import CoreNLPParser
from nltk.parse.corenlp import CoreNLPDependencyParser
import en, re
from sent_processor import sent_converter


def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    # A way to preserve ordering
    s1_spl = s1.split(" ")
    s2_spl = s2.split(" ")
    order = 0
    # Prioritize matching in order
    for sp in s2_spl:
        while len(s1_spl) and sp not in s1_spl[0]:
            s1_spl.pop(0)
        if len(s1_spl):
            order += 1
    return previous_row[-1] - order

def all_include(x1, x2):
    if len(x1) > len(x2):
        return all_include(x2, x1)
    for x in x1:
        if x not in x2:
            return False
    return True

class parser:
    def __init__(self, utterance):
        self.parser_core = CoreNLPParser(url='http://localhost:9000')
        self.dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')
        self.ner_parser = CoreNLPParser(url='http://localhost:9000', tagtype='ner')
        self.sentence = utterance
        # self.shift_sent()
        # self.sentence = self.removePunct()
        self.parseTree = self.parser_core.parse(self.sentence.split())
        self.dep_tr = self.dep_parser.raw_parse(self.sentence)
        # POS tags / words associated with each verb
        self.verb_clusters = dict()
        self.preps = []
        self.verbs = []
        # POS structure / frame of the input
        self.frame = None
        self.parent_dict = dict()
        self.voc_dict = dict()
        self.parse_voc_dict(self.parseTree)
        if len(self.verbs) == 0:
            return
        self.parse_dep()


    def removePunct(self):
        # remove all punctuation except periods & separate numbers
        original_sent = self.sentence.lower()
        # original_sent = self.sentence
        original_sent = original_sent.replace("!", ".")
        original_sent = original_sent.replace("?", ".")
        original_sent = original_sent.replace("-", " ")
        original_sent = original_sent.replace("mr.", "mr")
        original_sent = original_sent.replace("ms.", "ms")
        original_sent = original_sent.replace("mrs.", "mrs")
        original_sent = original_sent.replace("dr.", "dr")
        original_sent = original_sent.replace("drs.", "drs")
        original_sent = original_sent.replace("st.", "st")
        original_sent = original_sent.replace("sgt.", "sgt")
        original_sent = original_sent.replace("lt.", "lt")
        original_sent = original_sent.replace("fr.", "fr")
        original_sent = original_sent.replace("pp.", "pp")
        original_sent = original_sent.replace("pg.", "pg")
        original_sent = original_sent.replace("pgs.", "pgs")
        original_sent = original_sent.replace("pps.", "pps")
        original_sent = original_sent.replace(".com", " com")
        original_sent = original_sent.replace(".net", " net")
        original_sent = original_sent.replace(".edu", " edu")
        original_sent = original_sent.replace("www.", "www ")
        original_sent = original_sent.replace("...", " ")
        original_sent = original_sent.replace("$", "$ ")
        original_sent = original_sent.replace("%", " % ")
        original_sent = re.sub('[@#^&*\(\)_\+`\-=\\/,<>:;\"\.]+', " ", original_sent)
        original_sent = original_sent.replace("'s ", " s ")
        original_sent = original_sent.replace("s' ", "s ")
        original_sent = original_sent.replace("'d ", " would ")
        original_sent = original_sent.replace("'ll ", " will ")
        original_sent = original_sent.replace("'ve ", " have ")
        original_sent = original_sent.replace(" they're ", " they are ")
        original_sent = original_sent.replace(" we're ", " we are ")
        original_sent = original_sent.replace("'t ", " t ")
        # original_sent = original_sent.replace("o'clock", "o clock")
        # original_sent = re.sub("(?<![a-zA-Z]{2,})\.","",original_sent,flags=re.IGNORECASE) #removes periods in initials
        while "  " in original_sent:
            original_sent = original_sent.replace("  ", " ")
        return original_sent

    def split_sent(self):
        sent = self.sentence
        sent = re.sub('[@#^&*\(\)_\+`=\\/,<>;:\"\.]+', ".", sent)
        sent = sent.strip("\n")
        return sent

    # get the top-level parts of speech
    def getPOSs(self, tree):
        """
        Obtain the top-level parts of speech tags. Fill voc_dict with the tags and their corresponding words.
        e.g. {"NP-1": "Captain Kirk"}
        :param tree: the parse tree of the original sentence
        :return: the list of top-level POS tags
        """
        poslist = []
        for subtree in tree:
            if type(subtree) == nltk.tree.Tree:
                if subtree.label() == 'PP' or subtree.label() == 'S' or subtree.label() == 'NP' or subtree.label() == 'ADVP':
                    if subtree.label() == 'PP':
                        # if the first word is not in prepositions, change to NP
                        tag = pos_tag(word_tokenize(subtree.leaves()[0]))
                        if tag[0][1] == 'IN' or tag[0][1] == 'TO':
                            self.preps.append(subtree.leaves()[0])
                        else:
                            subtree.set_label('NP')
                    elif subtree.label() == 'ADVP':
                        # If there is an ADVP, add the ADV as well
                        import copy
                        sub_1 = copy.deepcopy(subtree)
                        sub_1.set_label('ADV')
                        self.voc_dict.update({sub_1.label() + "-" + str(len(self.voc_dict) + 1): " ".join(sub_1.leaves())})
                        poslist += [sub_1]
                    self.voc_dict.update({subtree.label() + "-" + str(len(self.voc_dict) + 1): " ".join(subtree.leaves())})
                    poslist += [subtree]  # add this pos subtree to the list and stop
                elif 'VB' in subtree.label():
                    poslist += ['V']
                    self.voc_dict.update(
                        {"V-" + str(len(self.voc_dict) + 1): " ".join(subtree.leaves())})
                    self.verbs.append(en.verb.present(subtree.leaves()[0]))
                elif subtree.label() == 'SBAR':
                    if "that" in subtree.leaves():
                        poslist += ["that"]
                        poslist += self.getPOSs(subtree)
                    elif "whether" in subtree.leaves():
                        poslist += ["whether"]
                        poslist += self.getPOSs(subtree)
                    elif "what" in subtree.leaves():
                        poslist += ["what"]
                        poslist += self.getPOSs(subtree)
                    elif "how" in subtree.leaves():
                        poslist += ["how"]
                        poslist += self.getPOSs(subtree)
                else:
                    poslist += self.getPOSs(subtree)
        return poslist

    def parse_voc_dict(self, parseT):
        # Try to find the highest parent for every word
        # This is a DFS
        # print("PARSE-T POS--------------")
        parsed_pos = []
        for t in parseT:
            for tr in t:
                parsed_sentence = self.getPOSs(tr)
                for parse in parsed_sentence:
                    if type(parse) is str:
                        parsed_pos += [parse]
                    else:
                        parsed_pos += [parse.label()]
        self.frame = parsed_pos
        # print(self.voc_dict)

    # def get_dep(self, dep_tree, dep_dict):
        

    def trav_dep(self, dep_tree, verbs, dep_dict, curr_v):
        # A depth first search in the dependency parse tree
        if isinstance(dep_tree, str):
            # Which words depend on the current verb
            dep_dict[curr_v].append(dep_tree)
            return dep_dict
        else:
            label = dep_tree.label()
            # Check if the current label is a verb
            try:
                verb = en.verb.present(label)
            except:
                verb = "Not Verb"
            if verb == 'Not Verb' or verb not in verbs:
                leaves = dep_tree.leaves()
                words = []
                for l in leaves:
                    try:
                        verb = en.verb.present(l)
                        words.append(en.verb.present(l))
                    except:
                        words.append("Not Verb")

            # "BUFFER" deals with the situation where a verb may appear after its dependents
            if verb in verbs:
                # The first actual verb would have the same dependents as the "BUFFER"
                if curr_v == 'BUFFER':
                    # Clear the buffer
                    dep_dict[verb].extend(dep_dict['BUFFER'])
                    dep_dict['BUFFER'] = []
                curr_v = verb
            elif curr_v is None:
                curr_v = 'BUFFER'
                dep_dict.update({curr_v: []})
            dep_dict[curr_v].append(label)
            for c in dep_tree:
                dep_dict = self.trav_dep(c, verbs, dep_dict, curr_v)
            # print("Dep Dict", dep_dict)
            return dep_dict


    def parse_dep(self):
        tree = None
        # The dictionary of subjects; fill the verb's dependent with the subjects in the sentence
        subj_dict = dict()
        # Put in all the verbs
        for t in self.dep_tr:
            tree = t
            dep_dict = dict()
            # Each verb has a series of dependents
            for k in self.verbs:
                dep_dict.update({k: []})
            self.verb_clusters = self.trav_dep(t.tree(), self.verbs, dep_dict, None)
            # This is where you match the clusters

            for governor, dep, dependent in tree.triples():
                # If two words are conj, they have the same nsubj
                if (dep == 'nsubj' or dep == 'nsubjpass') and ('VB' in governor[1]):
                    subj_dict.update({en.verb.present(governor[0]): dependent[0]})
        for v in self.verbs:
            temp = []
            cluster = self.verb_clusters[v]
            for label in self.voc_dict:
                if all_include(self.voc_dict[label].split(" "), cluster):
                    temp.append(label)
            self.verb_clusters[v] = temp

        for governor, dep, dependent in tree.triples():
            # If two words are conj, they have the same nsubj
            if (dep == 'conj') and ('VB' in governor[1]) and ('VB' in dependent[1]):
                try:
                    if en.verb.present(governor[0]) not in subj_dict:
                        v_app = en.verb.present(governor[0])
                        v_or = en.verb.present(dependent[0])
                    else:
                        v_app = en.verb.present(dependent[0])
                        v_or = en.verb.present(governor[0])
                    for l in self.voc_dict:
                        if subj_dict[v_or] in self.voc_dict[l] and v_app in self.verb_clusters:
                            self.verb_clusters[v_app].insert(0, l)
                            subj_dict.update({v_app: l})
                except KeyError:
                    return

        if len(subj_dict) == 1:
            # If there is only one subject, then that's it!
            subj = list(subj_dict.values())[0]
            for l in self.voc_dict:
                if subj in self.voc_dict[l]:
                    for v in self.verbs:
                        self.verb_clusters[v].insert(0, l)
        elif len(subj_dict) > 0:
            # If there are multiple subjects, pick the closest previous subject
            for i in range(1, len(self.verbs)):
                if self.verbs[i] not in subj_dict:
                    for l in self.voc_dict:
                        if subj_dict[self.verbs[i - 1]] in self.voc_dict[l]:
                            self.verb_clusters[self.verbs[i]].insert(0, l)
                            subj_dict.update({self.verbs[i]: l})

        for v in self.verb_clusters:
            self.verb_clusters[v] = list(dict.fromkeys(self.verb_clusters[v]))


# def ditch_unecessary(frame, fr_s):
#     # Compare against all frames, get rid of POS tags in the original frame that is not present in any of the candidates
#     fr = " ".join([f['description']['primary'] for f in fr_s])
#     fr = list(set(fr.split(" ")))
#     for i in range(len(fr)):
#         if "-" in fr[i]:
#             fr[i] = fr[i][:fr[i].index("-")]
#         if "." in fr[i]:
#             fr[i] = fr[i][:fr[i].index(".")]
#     frame = [f[:f.index("-")] for f in frame] if "-" in "".join(frame) else frame
#     new_frame = []
#     for f1 in frame:
#         if f1 in fr:
#             new_frame.append(f1)
#     return new_frame

import copy
def count_fill(orig, frame):
    len_frame = len(frame)
    num_fill = 0
    for o in orig:
        while len(frame) and frame[0] != o:
            frame.pop(0)
        num_fill += 1
    return float(num_fill)/len_frame



def parse_all(utterance, total_count=0):
    """
    Parse through a given sentence and match it to a VerbNet frame and fill the frame
    :param utterance:
    :param total_count: controls if we will edit the sentence and try to parse it again: if total_count is 0, we would edit if necessary; if total_count is 1, we would not edit
    :return: found (if there is a valid frame), the entire filled frame, and prepositions in the sentence
    """
    p = parser(utterance)
    frame_dict = dict()
    frame_dict_total = dict()
    fd = False

    if len(p.voc_dict) == 0:
        return (False, frame_dict, [])


    # If starting with Prep Phrase, the sentence is in inverted order
    # Need to pass into sent converter for it to be in the natural order
    if ("PP-" in list(p.voc_dict.keys())[0]) and total_count == 0:
        p1 = sent_converter()
        s1 = p1.conv_sent(p.sentence)
        return parse_all(s1, total_count=1)


    # If the top-level POS's are all sentences --> The whole sentence is made of all sub-sentences
    # Then parse each sub-sentence and return the combined result
    if len([x for x in p.voc_dict.keys() if "S-" in x]) == len(p.voc_dict) and len(p.voc_dict) > 0 and total_count == 0:
        for s in p.voc_dict.values():
            found, f_dict, preps = parse_all(s, total_count=1)
            if preps is not None:
                p.preps.extend(preps)
            fd = fd | found
            if f_dict is not None:
                for k, v in f_dict.items():
                    frame_dict.update({k: v})
        return (fd, frame_dict, p.preps)

    if p.frame == 'EMPTY_FRAME':
        return (False, None, None)

    import pickle, json
    # The mapping file has the verb and its matching class id's
    f = open('dictionaries/mapping_v.p', 'rb')
    mapping = pickle.load(f)

    # Get Verbnet
    f = open("dictionaries/verbnet.json", 'r')
    f_c = json.load(f)
    v_n = f_c['VerbNet']

    _syntax = []
    _semantics = []
    prep_total = dict()
    # Traverse verbs and find all the frames
    for ind, verb in enumerate(p.verbs):
        prep_match = dict()
        gen_match = dict()
        frame_dict = dict()
        potentials = []
        prep_total.update({verb: []})
        try:
            class_ids = mapping[verb]
        except KeyError:
            print("Cannot find verb in mapping!")
            return (False, None, None)
        for v in v_n:
            max_score = 100
            if v['class_id'] in class_ids:
                frame = p.verb_clusters[verb]
                ori_frame = p.verb_clusters[verb]
                fr_s = v['frames']
                for f in fr_s:
                    f.update({"CLASS": v['class_id']})
                    # Obtain frame description
                    f_str = f['description']['primary'].split(" ")
                    f_str = [x.split(".")[0].split("_")[0].split("-")[0] for x in f_str]
                    f_str = list(filter(None, f_str))
                    lev_score = levenshtein(" ".join(frame), " ".join(f_str))

                    # Only append the frame to the final result if can_append is true
                    can_append = True

                    # Preposition matches are more definite (i.e. if the preposition matches with the candidate, it is more likely that this candidate is correct)
                    if "PP" in " ".join(frame) and "PP" in f_str:
                        # Extract all prepositions from the candidate frame
                        vals = [args for args in f['syntax'] if args['arg_type'] == 'PREP' or args['arg_type'] == 'TO']
                        if len(vals) > 0:
                            # Maybe instead of directly p.preps, use individual prep sets instead?
                            if vals[0]['value'].islower() and len(set(p.preps).intersection(set(vals[0]['value'].split(" ")))) > 0:
                                prep_match.update({lev_score: f})
                            elif vals[0]['value'] == "PREP":
                                can_append = True
                            else:
                                can_append = False
                    if lev_score <= max_score:
                        potentials.append(f)
                        # print("NEW POTENTIAL MATCH ======")
                        # print(f['description']['primary'])
                        if lev_score == max_score:
                            # print(gen_match, max_score)
                            f_or = gen_match[max_score]['description']['primary'].split(" ")
                            # Filler metric
                            ##################################
                            # How many slots have been filled in order
                            sc_1 = count_fill(ori_frame, f_or)
                            sc_2 = count_fill(ori_frame, f_str)
                            can_append = can_append and (sc_1 < sc_2)
                        if can_append:
                            # This is a generic match
                            gen_match.update({lev_score: f})
                            max_score = lev_score
        # If there are no generic matches, we need to process the sentence and try to parse again
        if len(gen_match) == 0:
            if total_count == 0:
                p1 = sent_converter()
                s1 = p1.conv_sent(p.sentence)
                # print("Converted Sentence", s1)
                return parse_all(s1, total_count=1)
            # Select based on best match score??
        else:
            # Find the match with the lowest levenshtein score / best match
            if len(prep_match) and min(list(gen_match.keys())) == min(list(prep_match.keys())):
                print("Prep match!!!!")
                match = prep_match[min(list(prep_match.keys()))]
            else:
                match = gen_match[min(list(gen_match.keys()))]
            # Match the slots!!
            print("FINAL MATCH")
            print("Original Sentence:", p.sentence)
            print(match['description']['primary'])
            syntax = match['syntax']
            # THE S_INF SITUATION
            # Sometimes, the verbnet frame has S- (sentence), but the actual syntax does not have sentence
            # Main idea is to make all S- NP-
            syntaxes = [s['arg_type'] for s in syntax]
            if "S" in match['description']['primary'] and "S-" in " ".join(ori_frame) and "S" not in syntaxes:
                # If the lengths don't match
                s_s = [(x, p.voc_dict[x]) for x in p.voc_dict if "S-" in x]
                if len(match['description']['primary'].split()) - len(ori_frame) <= 2:
                    for k, s in s_s:
                        k1 = k.replace("S-", "NP-")
                        # update voc dict
                        # update verb cluster
                        new_voc = dict()
                        for v in p.voc_dict:
                            if v == k:
                                new_voc.update({k1: p.voc_dict[v]})
                                # Also replace everything in verb cluster
                                for n, l in p.verb_clusters.items():
                                    p.verb_clusters[n] = [k1 if x == v else x for x in l]
                            else:
                                new_voc.update({v: p.voc_dict[v]})
                        p.voc_dict = new_voc

                else:
                    while "S-" in " ".join(list(p.voc_dict.keys())):
                        # The S_INF situation
                        for k, s in s_s:
                            if s.split()[0] == 'to':
                                p.preps.append("to")
                                s = " ".join(s.split()[1:])
                            p1 = parser(s)
                            new_voc = dict()
                            # new_vc = dict()
                            for v in p1.voc_dict:
                                new_voc.update({v + "_" + str(len(p.voc_dict)): p1.voc_dict[v]})
                                # Also replace everything in verb cluster
                                for n, l in p1.verb_clusters.items():
                                    p1.verb_clusters[n] = [v + "_" + str(len(p.voc_dict)) if x == v else x for x in l]
                            p1.voc_dict = new_voc
                            if len(p1.voc_dict) == 1:
                                # need to split further
                                pt = p1.parser_core.parse(list(p1.voc_dict.values())[0].split())
                            for k1, v1 in p1.voc_dict.items():
                                p.voc_dict.update({k1: v1})
                            p.voc_dict.pop(k)
                            p.verbs.extend(p1.verbs)
                            for k1, v1 in p1.verb_clusters.items():
                                p.verb_clusters.update({k1: v1})
                            #TODO: substitute the original sentence with all the voc
                            for k1, v1 in p.verb_clusters.items():
                                if k in v1:
                                    # replace everything
                                    for n_v in list(p1.voc_dict.keys()):
                                        v1.insert(v1.index(k), n_v)
                                    v1.remove(k)
                                    p.verb_clusters[k1] = v1


            pair_list = [(x[:x.index("-")], p.voc_dict[x]) for x in p.verb_clusters[verb]]
            fill_prep = False
            frame_dict.update({"FRAME": match['description']['primary']})

            prep_list = []

            # If there is Prep, do the conversion
            # Substitute PP = Prep + NN
            if "PP-" in " ".join(ori_frame):
                le = [x for x in p.verb_clusters[verb] if "PP-" in x]
                for l in le:
                    pp = p.voc_dict[l]
                    prep = pp.split(" ")[0]
                    pp = " ".join(pp.split(" ")[1:])
                    for t in p.parser_core.raw_parse(pp):
                        spl = str(t).split("(")
                        spl = [s for s in spl if len(s) > 2 and "S " not in s and "FRAG" not in s and "ROOT" not in s]
                        new_label = spl[0].split(" ")[0]
                        new_label = new_label.strip()
                        if (new_label == 'NN') or (new_label == "S" and "S" not in [x['arg_type'] for x in syntax]):
                            new_label = 'NP'
                        # p.voc_dict.update({"Prep-" + new_label: pp})
                        prep_list.append(tuple(["Prep-" + new_label, prep, pp]))

            # S_ING --> NP
            # print("Pair List:", pair_list)
            # print("Prep List", prep_list)
            # Handle the case where there is PP in the verbnet frame, but no PREP in the syntax
            prep_copy = copy.copy(prep_list)
            print("prep copy", prep_copy, "prep", prep_list)
            prep_dic = {'arg_type': 'PREP', 'value': 'PREP', 'selrestrs': {'selrestr_list': None, 'selrestr_logic': None}, 'synrestrs': {'synrestr_list': None, 'synrestr_logic': '&'}}
            if "PP" in match['description']['primary'] and "PREP" not in [s['arg_type'] for s in syntax]:
                # Find the index of the prep
                ori_copy = copy.copy(pair_list)
                frame_copy = match['description']['primary'].split(" ")
                ind = 0
                while len(ori_copy) and ori_copy[0][0] != 'PP':
                    up_to = ori_copy.pop(0)
                    while len(frame_copy) and frame_copy[0] != up_to:
                        ind += 1
                        frame_copy.pop(0)
                match['syntax'].insert(ind - 1, prep_dic)

            # Block out things that you cannot fill
            # Do set intersection
            ori_copy = copy.deepcopy(pair_list)
            fill_prep = False
            for s in syntax:
                arg = s['arg_type']
                # Compare with the 0th element
                if (len(ori_copy) == 0) or (not ((ori_copy[0][0] == arg) or (arg == "VERB" and ori_copy[0][0] == "V") or (arg == "PREP" and ori_copy[0][0] == "PP") or ("Prep-" +  arg in [x[0] for x in prep_copy] and fill_prep))):
                    s['arg_type'] = "?"
                elif arg == "PREP" and ori_copy[0][0] == "PP":
                    fill_prep = True
                elif "Prep-" +  arg in [x[0] for x in prep_copy] and fill_prep:
                    fill_prep = False
                    prep_copy.pop(0)
                # if len(ori_copy) and ori_copy[0][0] != "PP":
                if len(ori_copy):
                    ori_copy.pop(0)
                # Keep popping
                # Handle the case of VERB
                # Handle the case of PREP + NP

            print("prep copy", prep_copy, "prep", prep_list)
            # Fill things that are not ?
            for s in syntax:
                if s['arg_type'] == 'VERB':
                    s['arg_type'] = verb
                    frame_dict.update({s['value']: verb})
                    # Pop out the non-verbs before "V" on pair_list
                    while len(pair_list) and pair_list[0][0] != 'V':
                        pair_list.pop(0)
                elif s['arg_type'] == 'PREP' and "PP-" in " ".join(ori_frame) and len(prep_list):
                    while len(pair_list) and pair_list[0][0] != 'PP':
                        pair_list.pop(0)
                    fill_prep = True
                    s['arg_type'] = prep_list[0][1]
                    frame_dict.update({s['value']: prep_list[0][1]})
                elif s['arg_type'] != "?":
                    if fill_prep and "PP-" in " ".join(ori_frame):
                        s['arg_type'] = prep_list[0][2]
                        prep_list.pop(0)
                        fill_prep = False
                    else:
                        rm_ind = -1
                        filled = False
                        for i in range(len(pair_list)):
                            k, v = pair_list[i]
                            if k in s['arg_type'] and not filled:
                                s['arg_type'] = v
                                frame_dict.update({s['value']: v})
                                rm_ind = i
                                filled = True
                        if rm_ind != -1:
                            del pair_list[rm_ind]
            # Fill the slots
            semantics = match['semantics']
            print(semantics)
            for pred in semantics:
                for a in pred['args']:
                    sy_match = [sy['arg_type'] for sy in syntax if sy['value'] == a['value']]
                    if len(sy_match) > 0:
                        a['value'] = sy_match[0]
            frame_dict.update({"SYNTAX": syntax})
            frame_dict.update({"SEMANTICS": semantics})
            frame_dict.update({"ENTIRE_FRAME": match})
            _syntax.append(syntax)
            _semantics.append(semantics)
        frame_dict_total.update({verb: frame_dict})
    # NER parse utterance
    ner_list = list(p.ner_parser.tag((utterance.split())))
    # print(ner_list)
    types = ""
    names = ""
    for ne in ner_list:
        if ne[1] != 'O':
            types = types + ne[1] + ","
            names = names + ne[0] + " "
        else:
            types = types + "|"
            names = names + "|"
    ner_list = []
    names = [x.strip() for x in names.split("|") if len(x) > 0]
    types = [x for x in types.split("|") if len(x) > 0]
    # print("NAMES AND TYPES", names, types)
    for i in range(len(types)):
        types[i] = types[i].split(",")[0]
        if types[i] == "PERSON":
            ner_list.append(("PERSON", names[i]))
        elif types[i] == "TITLE":
            ner_list.append(("PERSON", "the " + names[i]))
        else:
            ner_list.append((types[i], names[i]))
    ner_list.sort(key=lambda t: len(t[1]), reverse=True)
    frame_dict_total.update({"NER": ner_list})
    return (True, frame_dict_total, p.preps)



In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "You are dissecting provided news article headlines into the following form: (Agent, Recipient,  Action, Object, Emotion_A, Emotion_R) where agent is the actor, recipient is the reactor, action is what is being carried out, object is what it's carried out on, and emotions of A and R are the emotions of the agent and recipient respectively,  and pull emotions from the pool of: Sad, Happy, Excited, Tender, Angry,  Scared. Display your chain of thought in <thinking> tags for the emotions alone. Display ONLY the bracket set and your thinking process.",
          },
          {
              "type":"text",
              "text":"Insert news article here"
          }
        ]
      }
    ],
    "max_tokens": 500
 }
  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  print(response.json())
  with open('part1.json', 'w') as file:
      file.write(json.dumps(response.json(), indent=4))
  with open('part1.json', 'r') as file:
      data = json.load(file)
      resp_array.append(data['choices'][0]['message']['content'])
  with open("story_coherence.txt",'a') as file:
      file.write(data['choices'][0]['message']['content']+"\n\n\n")